In [163]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import matplotlib.path as mpath
import pandasql as sql
import collections
from datetime import datetime

In [164]:
ms1 = pd.read_excel("C:/Users/NImi Jithin/Desktop/data/measurements/ms1.xlsx")

In [165]:
len(ms1)

5771

In [166]:
ms1.head(5)

,time,lat,long,dir,velocity,T_outside,brightnessFW,brightnessIR,PredictedSeg
0,"Wednesday, November 1, 2017 16:17:31:83508813",51.900768,10.43236,0.0,0.0,12.5,827.398613,2000.0,6471
1,"Wednesday, November 1, 2017 16:17:32:33509545",51.900768,10.43236,0.0,0.0,12.5,827.398613,2000.0,6471
2,"Wednesday, November 1, 2017 16:17:32:83509277",51.900768,10.43236,0.0,0.0,12.5,828.000000,2000.0,6471
3,"Wednesday, November 1, 2017 16:17:33:33509008",51.900768,10.43236,0.0,0.0,12.5,827.395594,2000.0,6471
4,"Wednesday, November 1, 2017 16:17:33:83508715",51.900768,10.43236,0.0,0.0,12.5,824.368410,2000.0,6471


In [167]:
ms1['id'] = ms1.index

In [168]:
q1 = """SELECT PredictedSeg,COUNT(*) as count FROM ms1 GROUP BY PredictedSeg ORDER BY PredictedSeg """
test = sql.sqldf(q1, locals())

In [169]:
print(ms1.dtypes) # dataype

time             object
lat             float64
long            float64
dir             float64
velocity        float64
T_outside       float64
brightnessFW    float64
brightnessIR    float64
PredictedSeg      int64
id                int64
dtype: object


In [170]:
tot_len = len(ms1.iloc[0]['time']) # TODO : need to generalise

In [171]:
start, stop, step = 0, tot_len-2, 1
ms1['time'] = ms1["time"].str.slice(start, stop, step) 

In [172]:
ms1['time'] = ms1['time'].apply(lambda x: datetime.strptime(x, '%A, %B %d, %Y %H:%M:%S:%f'))

In [173]:
######################################################## TODO : Need to automate this
#ms1['Day'] = 'Wednesday'
#ms1['Month'] = 'November'
the_date1 = ms1.iloc[1]['time']
the_date2 = ms1.iloc[2]['time']
diff = the_date2 - the_date1
print(diff.total_seconds())

0.499997


In [175]:
i=0
mesured_order = 0;
df = pd.DataFrame({'Start_Time': [], 'Duration': [], 'Duration': [], 'Predicted_Seg': []})
while(i<len(ms1)):
    predicted_seg = ms1.iloc[i]['PredictedSeg']
    aggregate_velocity = 0
    i = i+1
    for j in range(i, len(ms1)):
        if ms1.iloc[j]['PredictedSeg'] != predicted_seg:
            previous_i = i
            i=j
        else:
            time_diff = ms1.iloc[j]['time']-ms1.iloc[j-1]['time']
            aggregate_velocity = aggregate_velocity + ms1.iloc[j-1]['velocity'] * time_diff.total_seconds()
            continue
        
        duration = ms1.iloc[j]['time'] - ms1.iloc[previous_i]['time']
        df = df.append({'mesured_order': mesured_order,
                        'Start_Time': ms1.iloc[previous_i-1]['time'], 
                        'End_Time': ms1.iloc[j]['time'],
                        'Duration': duration.total_seconds(), 
                        'Agg_Velocity': aggregate_velocity,
                        'Predicted_Seg': predicted_seg},ignore_index=True)
        
        mesured_order = mesured_order + 1
        break 
        
    if j == len(ms1)-1:
        break
    
        


In [176]:
df.head(5)

,Start_Time,Duration,Predicted_Seg,Agg_Velocity,End_Time,mesured_order
0,2017-11-01 16:17:31.835088,37.499988,6471.0,255.309701,2017-11-01 16:18:09.835083,0.0
1,2017-11-01 16:18:09.835083,2.499996,6542.0,76.239341,2017-11-01 16:18:12.835086,1.0
2,2017-11-01 16:18:12.835086,3.500001,6555.0,74.918134,2017-11-01 16:18:16.835085,2.0
3,2017-11-01 16:18:16.835085,3.500011,6606.0,57.275904,2017-11-01 16:18:20.835093,3.0
4,2017-11-01 16:18:20.835093,1.500002,6599.0,38.320467,2017-11-01 16:18:22.835093,4.0
